# Machine Learning Algorithms Comparitive Analysis for prediction of Diabetes


### Importing Libraries


In [ ]:
import os

os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"

In [ ]:
# Importing Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential, clone_model
from keras.layers import Dense, Dropout

# from scikeras.wrappers import KerasClassifier
import sklearn as sk
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import seaborn as sns

### Importing Dataset


In [ ]:
df = pd.read_csv("../../Dataset/diabetes_data_upload.csv")
df.head()

In [ ]:
# Checking for null values
df.isna().sum()

In [ ]:
# Checking for data types
df.info()

# Coverting columns to lower case
df.columns = map(str.lower, df.columns)

### Data Visualization


In [ ]:
# Diabetes distribution by age
plt.figure(figsize=(8, 6))
sns.histplot(df["age"], bins=30, color="royalblue", kde=True)
plt.title("Diabetes distribution by age", fontsize=16)

In [ ]:
# Diabetes distribution by gender
plt.title("Diabetes cases by Gender", fontsize=17)
sns.countplot(x="gender", hue="class", data=df, palette=["#ff8080", "#78ebdc"])

In [ ]:
# Bar plots for symptoms and diabetes cases
count = 1
plt.figure(figsize=(15, 20))
plt.suptitle("Correlation between Symptoms and Diabetes Cases" + "\n", fontsize=20)
for i in df.columns:
    if i not in ["class", "age", "gender"]:
        plt.subplot(5, 4, count)
        plt.title(f"{i.title()}", fontweight="bold", fontsize=14)
        count += 1
        plt.tight_layout()
        total_count = df[i].value_counts()
        total_percentage = total_count / total_count.sum() * 100
        total_percentage.plot(kind="bar", color=["royalblue", "lightblue"])
        plt.ylabel("Count")
        plt.title(f"{i.title()}", fontweight="bold", fontsize=12)
        for j, v in enumerate(total_percentage):
            plt.text(j, v, f"{v:.1f}%", ha="center", va="bottom", fontweight="bold")
plt.show()

### Data Preprocessing


In [ ]:
# Changing positive to 1 and negative to 0
df["class"] = df["class"].replace({"Positive": 1, "Negative": 0})

# Seperating target feature from dataset
X = df.drop("class", axis=1)
y = df["class"]

# Storing features to list
objectList = X.select_dtypes(include="object").columns.tolist()
print(objectList)

In [ ]:
# Label Encoding for object to numeric transformation
encoder = preprocessing.LabelEncoder()

for col in X.columns:
    if X[col].dtype == "object":
        X[col] = encoder.fit_transform(X[col])

# Printing datatypes of features
X.info()

In [ ]:
# Printing first 5 rows of dataset
X.head()

In [ ]:
# Correlation between features and diabetes
X.corrwith(y).plot(
    kind="bar", grid=True, figsize=(8, 4), title="Correlation with Diabetes"
)

In [ ]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=40
)

# Print the shapes of the training and testing sets
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

### Data Normalization


In [ ]:
minmax = MinMaxScaler()
X_train[["age"]] = minmax.fit_transform(X_train[["age"]])
X_test[["age"]] = minmax.transform(X_test[["age"]])
X_train.head()

### Model Building


##### Logistic Regression


In [ ]:
lg = LogisticRegression(random_state=0, penalty="l2")
lg.fit(X_train, y_train)

In [ ]:
# Cross Validation
kfold = model_selection.KFold(n_splits=10, random_state=7, shuffle=True)
score = "accuracy"

acc_lg = model_selection.cross_val_score(lg, X_train, y_train, cv=kfold, scoring=score)
acc_lg.mean()

In [ ]:
# Evaluation Metrics
y_pred_lg = lg.predict(X_test)

lg_acc = accuracy_score(y_test, y_pred_lg)
lg_roc = roc_auc_score(y_test, y_pred_lg)
lg_prec = precision_score(y_test, y_pred_lg)
lg_rec = recall_score(y_test, y_pred_lg)
lg_f1 = f1_score(y_test, y_pred_lg)

results_lg = pd.DataFrame(
    [["Logistic Regression", lg_acc, acc_lg.mean(), lg_roc, lg_prec, lg_rec, lg_f1]],
    columns=[
        "Model",
        "Accuracy",
        "Cross Val Accuracy",
        "ROC",
        "Precision",
        "Recall",
        "F1 Score",
    ],
)

results_lg

##### Decision Trees Classifier


In [ ]:
dt = DecisionTreeClassifier(criterion="gini", random_state=0)
dt.fit(X_train, y_train)

In [ ]:
# Cross Validation
acc_dt = model_selection.cross_val_score(dt, X_train, y_train, cv=kfold, scoring=score)
acc_dt.mean()

In [ ]:
# Evaluation Metrics
y_pred_dt = dt.predict(X_test)

dt_acc = accuracy_score(y_test, y_pred_dt)
dt_roc = roc_auc_score(y_test, y_pred_dt)
dt_prec = precision_score(y_test, y_pred_dt)
dt_rec = recall_score(y_test, y_pred_dt)
dt_f1 = f1_score(y_test, y_pred_dt)

results_dt = pd.DataFrame(
    [
        [
            "Decision Trees Classifier",
            dt_acc,
            acc_dt.mean(),
            dt_roc,
            dt_prec,
            dt_rec,
            dt_f1,
        ]
    ],
    columns=[
        "Model",
        "Accuracy",
        "Cross Val Accuracy",
        "ROC",
        "Precision",
        "Recall",
        "F1 Score",
    ],
)

results_dt

##### Random Forest Classifier


In [ ]:
estimators = 100

for i in range(1, estimators):
    rf = RandomForestClassifier(n_estimators=i, criterion="entropy", random_state=0)
    rf.fit(X_train, y_train)

In [ ]:
# Cross Validation
acc_rf = model_selection.cross_val_score(rf, X_train, y_train, cv=kfold, scoring=score)
acc_rf.mean()

In [ ]:
# Evaluation Metrics
y_pred_rf = rf.predict(X_test)

rf_acc = accuracy_score(y_test, y_pred_rf)
rf_roc = roc_auc_score(y_test, y_pred_rf)
rf_prec = precision_score(y_test, y_pred_rf)
rf_rec = recall_score(y_test, y_pred_rf)
rf_f1 = f1_score(y_test, y_pred_rf)

results_rf = pd.DataFrame(
    [
        [
            "Random Forest Classifier",
            rf_acc,
            acc_rf.mean(),
            rf_roc,
            rf_prec,
            rf_rec,
            rf_f1,
        ]
    ],
    columns=[
        "Model",
        "Accuracy",
        "Cross Val Accuracy",
        "ROC",
        "Precision",
        "Recall",
        "F1 Score",
    ],
)

results_rf

##### Neural Networks


In [ ]:
def create_model():
    model = Sequential(
        [
            Dense(16, input_dim=16, activation="relu"),
            Dropout(0.5),
            Dense(8, activation="relu"),
            Dropout(0.5),
            Dense(1, activation="sigmoid"),
        ]
    )
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model


nn = create_model()
nn.fit(X_train, y_train, epochs=100, batch_size=10, validation_split=0.2, verbose=0)

In [ ]:
# # Function to create model, required for KerasClassifier
# def create_model():
#     model = Sequential()
#     model.add(Dense(12, input_dim=16, activation='relu'))
#     model.add(Dense(8, activation='relu'))
#     model.add(Dense(1, activation='sigmoid'))
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return model

# # Wrap Keras model with KerasClassifier
# nn = KerasClassifier(build_fn=create_model, epochs=100, batch_size=5, verbose=0)

# nn.fit(X_train, y_train)

In [ ]:
# Custom cross-validation
def custom_cross_val_score(model_func, X, y, n_splits=10):
    kfold = model_selection.StratifiedKFold(
        n_splits=n_splits, shuffle=True, random_state=7
    )
    scores = []

    for train_ix, test_ix in kfold.split(X, y):
        model = clone_model(model_func())
        model.compile(
            optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
        )
        X_train_fold, X_test_fold = X[train_ix], X[test_ix]
        y_train_fold, y_test_fold = y[train_ix], y[test_ix]
        model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=10, verbose=1)
        scores.append(model.evaluate(X_test_fold, y_test_fold, verbose=0)[1])

    return np.mean(scores)


acc_nn = custom_cross_val_score(create_model, X_train.values, y_train.values)

In [ ]:
# Evaluation Metrics
y_pred_nn = nn.predict(X_test).ravel()
y_pred_nn_classes = (y_pred_nn > 0.5).astype(int)

nn_acc = accuracy_score(y_test, y_pred_nn_classes)
nn_roc = roc_auc_score(y_test, y_pred_nn)
nn_prec = precision_score(y_test, y_pred_nn_classes)
nn_rec = recall_score(y_test, y_pred_nn_classes)
nn_f1 = f1_score(y_test, y_pred_nn_classes)

results_nn = pd.DataFrame(
    [["Neural Network", nn_acc, acc_nn, nn_roc, nn_prec, nn_rec, nn_f1]],
    columns=[
        "Model",
        "Accuracy",
        "Cross Val Accuracy",
        "ROC",
        "Precision",
        "Recall",
        "F1 Score",
    ],
)

results_nn

##### K-Nearest Neighbor


In [ ]:
knn = KNeighborsClassifier(n_neighbors=5, metric="minkowski", p=2)
knn.fit(X_train, y_train)

In [ ]:
# Cross Validation
acc_knn = model_selection.cross_val_score(
    knn, X_train, y_train, cv=kfold, scoring=score
)
acc_knn.mean()

In [ ]:
# Evaluation Metrics
y_pred_knn = knn.predict(X_test)

knn_acc = accuracy_score(y_test, y_pred_knn)
knn_roc = roc_auc_score(y_test, y_pred_knn)
knn_prec = precision_score(y_test, y_pred_knn)
knn_rec = recall_score(y_test, y_pred_knn)
knn_f1 = f1_score(y_test, y_pred_knn)

results_knn = pd.DataFrame(
    [
        [
            "K-Nearest Neighbors",
            knn_acc,
            acc_knn.mean(),
            knn_roc,
            knn_prec,
            knn_rec,
            knn_f1,
        ]
    ],
    columns=[
        "Model",
        "Accuracy",
        "Cross Val Accuracy",
        "ROC",
        "Precision",
        "Recall",
        "F1 Score",
    ],
)

results_knn

In [ ]:
# Initialize a DataFrame to store the results
final_results = pd.DataFrame(
    columns=[
        "Model",
        "Accuracy",
        "Cross Val Accuracy",
        "ROC",
        "Precision",
        "Recall",
        "F1 Score",
    ]
)

# List of models and their names
models = [lg, dt, rf, nn, knn]
model_names = [
    "Logistic Regression",
    "Decision Trees Classifier",
    "Random Forest Classifier",
    "Neural Networks",
    "KNN",
]

for model, name in zip(models, model_names):
    # Predict the test set results
    y_pred = model.predict(X_test)

    if name == "Neural Networks":
        y_pred = (y_pred > 0.5).astype(int)

    # Calculate the evaluation metrics
    acc = accuracy_score(y_test, y_pred) * 100
    roc = roc_auc_score(y_test, y_pred) * 100
    prec = precision_score(y_test, y_pred) * 100
    rec = recall_score(y_test, y_pred) * 100
    f1 = f1_score(y_test, y_pred) * 100

    # Append the results to the DataFrame
    final_results = pd.concat(
        [
            final_results,
            pd.DataFrame(
                {
                    "Model": [name],
                    "Accuracy": [acc],
                    "Cross Val Accuracy": [acc.mean()],
                    "ROC": [roc],
                    "Precision": [prec],
                    "Recall": [rec],
                    "F1 Score": [f1],
                }
            ),
        ],
        ignore_index=True,
    )

final_results.plot(
    x="Model",
    y="Accuracy",
    kind="bar",
    title="Comparison of Model Accuracies",
    xlabel="Models",
    rot=20,
)
plt.ylabel("Accuracy", rotation=0, ha="right")
plt.gca().get_legend().remove()

# Add percentage labels to the bars
for i, v in enumerate(final_results["Accuracy"]):
    plt.text(i, v, f"{v:.2f}", ha="right", va="bottom")

# Print the final results
final_results